## Multi-Label Chest X-Ray Pathology Detection of COVID-19 with Transfer Learning
Colm Clancy : 01365631 


### Metadata Extraction and Dataset Creation

In [ ]:
'''
#load padchest file list
rootdir_padchest = 'E:\\osfstorage-archive\\all_images\\'
E:\BIMCV-PadChest-FULL\0
p_list = list()
import os
for root, dirs, files in os.walk(rootdir_padchest):
    for name in files:
        if name.endswith((".json")) and name.startswith(("sub")) :
            p_list.append(name)
'''

In [ ]:
##### copy selected images to directory
#dir_1 =  'E:/osfstorage-archive/C19_images/'
#file_list = list(metadata_dummies.filename)
import shutil
'''
i=0
while i < len(file_list):
          shutil.copy(dir_1+file_list[i], 'E:/osfstorage-archive/C19_images/final/')
          i += 1


dir_1_pc2 =  'E:/BIMCV-PadChest-FULL/0/'
file_list_pc2 = list(pc2.filename)

i=0
while i < len(file_list_pc2):
          shutil.copy(dir_1_pc2+file_list_pc2[i], 'E:/osfstorage-archive/C19_images/final/')
          i += 1

'''

### Load BIMCV COVID-19+ metadata from BIMCV_metadata noteebook

In [2]:
import pandas as pd
import os
import json
#import warnings
#warnings.filterwarnings('ignore')

In [1]:
#load list of manually checked files : BIMCV Dataset filtered earlier
manual = pd.read_csv('E:\\manual_list.csv')
manual.head(1)

NameError: name 'pd' is not defined

In [ ]:
#load json file list
rootdir = 'E:\\osfstorage-archive\\all_images\\'
j_list = list()
import os
for root, dirs, files in os.walk(rootdir):
    for name in files:
        if name.endswith((".json")) and name.startswith(("sub")) :
            j_list.append(name)

In [ ]:
#add json files to json list
dallas = []

for x in j_list:
        with open('{}{}'.format(rootdir, x)) as i:
            dallas.append(json.load(i))

In [ ]:
#extract requied fields from json metadata files 
colour = []
position = []

for x in range(len(dallas)):
    try:
        colour.append(dallas[x]['00280004']['Value'])
        position.append(dallas[x]['00185101'])
    except KeyError:
         position.append('NA')

#create json_metadata dataframe            
import pandas as pd
json_metadata = pd.DataFrame(list(zip(colour,position,j_list)), 
               columns =['colour2','position','filename_json']) 


In [ ]:
json_metadata.to_csv('E:/json_metadata.csv')

In [ ]:
json_metadata.to_csv('json_metadata.csv')

In [ ]:
len(j_list)

In [ ]:
json_metadata.head(1)

In [ ]:
#replace .json with .png to help match
json_metadata['filename']= json_metadata['filename_json'].str.replace(r'.json', '.png')

#remove brackets from monochrome
json_metadata['colour2'] = json_metadata['colour2'].astype(str)
json_metadata['colour2'] = json_metadata['colour2'].str.replace("[","")
json_metadata['colour2'] = json_metadata['colour2'].str.replace("]","")

In [ ]:
json_metadata[json_metadata.filename == 'sub-S03085_ses-E06565_run-1_bp-chest_vp-ap_cr.png']

In [ ]:
json_metadata.head(1)

In [4]:
manual = manual[(manual.view == 'front')]
manual.shape


(2477, 7)

In [5]:
# no longer need json file data?
'''
man_and_json  = pd.merge(manual,json_metadata, left_on='img_list', right_on='filename', how = 'left' )
man_and_json_filter = man_and_json[(man_and_json.view == 'front')]
#man_and_json_filter = man_and_json[(man_and_json.colour == "'MONOCHROME2'") & (man_and_json.view == 'front')]

man_and_json_filter.head(6)
'''

'\nman_and_json  = pd.merge(manual,json_metadata, left_on=\'img_list\', right_on=\'filename\', how = \'left\' )\nman_and_json_filter = man_and_json[(man_and_json.view == \'front\')]\n#man_and_json_filter = man_and_json[(man_and_json.colour == "\'MONOCHROME2\'") & (man_and_json.view == \'front\')]\n\nman_and_json_filter.head(6)\n'

In [6]:
#man_and_json_filter.head(1)
#man_and_json_filter[man_and_json_filter.filename == 'sub-S03085_ses-E06565_run-1_bp-chest_vp-ap_cr.png']

In [7]:
# load label metadata
labels =  pd.read_csv('e:\\osfstorage-archive\\all_images\labels_covid19_posi.tsv', sep = '\t',engine='python')
labels.drop(columns=['Unnamed: 0', 'Report','Localizations','LabelsLocalizationsBySentence','labelCUIS','LocalizationsCUIS'],inplace=True)
labels.drop_duplicates(keep='first',inplace=True) 


In [8]:
#2404  2465
#31
#61
#labels.head(1)
#labels[labels.ReportID == 'ses-E06565']
labels.shape

(2434, 3)

In [9]:
# join label data to file list
#data_1  = pd.merge(man_and_json_filter,labels, left_on='ses', right_on='ReportID', how = 'left' )
data_1  = pd.merge(manual,labels, left_on='ses', right_on='ReportID', how = 'left' )

In [10]:
metadata = data_1[~data_1['ReportID'].isna()]
#102 images with no report

In [11]:
#data_1.drop(columns=['img_list','view','colour','position','filename_json','filename','sub','ses'],inplace=True)
metadata.head()

,img_list,view,colour,sub,ses,check_colour,check_view,PatientID,ReportID,Labels
0,sub-S03044_ses-E06138_run-1_bp-chest_vp-ap_cr.png,front,'MONOCHROME2',sub-S03044,ses-E06138,'MONOCHROME2',front,sub-S03044,ses-E06138,['endotracheal tube'\t 'central venous cathete...
1,sub-S03044_ses-E06790_run-1_bp-chest_vp-pa_dx.png,front,'MONOCHROME1',sub-S03044,ses-E06790,'MONOCHROME1',front,sub-S03044,ses-E06790,['chronic changes'\t 'increased density'\t ' p...
2,sub-S03044_ses-E07712_run-1_bp-chest_vp-ap_cr.png,front,'MONOCHROME2',sub-S03044,ses-E07712,'MONOCHROME2',front,sub-S03044,ses-E07712,['unchanged'\t 'central venous catheter via su...
3,sub-S03045_ses-E06139_run-1_bp-chest_vp-ap_dx.png,front,'MONOCHROME1',sub-S03045,ses-E06139,'MONOCHROME1',front,sub-S03045,ses-E06139,['interstitial pattern'\t 'cardiomegaly']
4,sub-S03045_ses-E06910_acq-1_run-1_bp-chest_vp-...,front,'MONOCHROME2',sub-S03045,ses-E06910,2,front,sub-S03045,ses-E06910,['atypical pneumonia'\t ' consolidation'\t ' p...


### Load padchest metadata

In [12]:
#load padchest data
pc = 'E:\\BIMCV-PadChest-FULL\\PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv\PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv'
padchest =  pd.read_csv(pc, sep = ',',engine='python')
padchest['StudyDate_DICOM'] = padchest['StudyDate_DICOM'].astype(str).str[0:4].astype(int)
padchest['age'] = padchest.StudyDate_DICOM - padchest.PatientBirth
padchest = padchest[padchest['PatientBirth'].notna()]


padchest = padchest[['ImageID','ImageDir','age','StudyID','PatientID','PatientSex_DICOM','Projection','Labels']]

#pc2 = padchest[padchest.ImageDir.le(0) & padchest.Projection.eq('PA') ]
pc2 = padchest[padchest.ImageDir.le(27) & padchest.Projection.isin(['PA','AP','AP_horizontal'])]

pc2.drop(['Projection','ImageDir'], axis=1, inplace=True)
pc2.rename(columns={'ImageID':'filename','StudyID':'ReportID','PatientSex_DICOM':'sex' }, inplace=True)
cn = ['filename','PatientID','ReportID','age','sex','Labels']
pc2 = pc2.reindex(columns=cn)

C:\Users\colmc\Anaconda3\envs\new_env\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\colmc\Anaconda3\envs\new_env\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
#pc2 = padchest[padchest.age.qe(54)
#padchest.age=84.0
pc2.shape

(58590, 6)

In [14]:
24384-23239

1145

In [15]:
#load age/sex
participants =  pd.read_csv('e:\\osfstorage-archive\\all_images\participants.tsv', sep = '\t',engine='python')
participants.rename(columns={'participant':'PatientID'}, inplace=True)
participants['age'] = participants['age'].str.replace("[","")
participants['age'] = participants['age'].str.replace("]","")
participants.age = participants.age.str[0:2]
participants.head(1)

,PatientID,modality_dicom,body_parts,age,gender
0,sub-S03852,['CR'],[['CHEST']],85,M


In [16]:
m2 = pd.merge(metadata,participants[['PatientID','age','gender']],on='PatientID', how='left')

In [17]:
m2.head(1)

,img_list,view,colour,sub,ses,check_colour,check_view,PatientID,ReportID,Labels,age,gender
0,sub-S03044_ses-E06138_run-1_bp-chest_vp-ap_cr.png,front,'MONOCHROME2',sub-S03044,ses-E06138,'MONOCHROME2',front,sub-S03044,ses-E06138,['endotracheal tube'\t 'central venous cathete...,67,M


In [18]:
#m2['img_list']= m2['img_list'].str.replace(r'.png', '.jpg')


In [19]:
m2.drop(['view','sub','view','colour','sub','ses','check_colour','colour','check_view'], axis=1, inplace=True)
m2.rename(columns={'gender':'sex', 'img_list':'filename'}, inplace=True)

In [20]:
m2.head(6)

,filename,PatientID,ReportID,Labels,age,sex
0,sub-S03044_ses-E06138_run-1_bp-chest_vp-ap_cr.png,sub-S03044,ses-E06138,['endotracheal tube'\t 'central venous cathete...,67,M
1,sub-S03044_ses-E06790_run-1_bp-chest_vp-pa_dx.png,sub-S03044,ses-E06790,['chronic changes'\t 'increased density'\t ' p...,67,M
2,sub-S03044_ses-E07712_run-1_bp-chest_vp-ap_cr.png,sub-S03044,ses-E07712,['unchanged'\t 'central venous catheter via su...,67,M
3,sub-S03045_ses-E06139_run-1_bp-chest_vp-ap_dx.png,sub-S03045,ses-E06139,['interstitial pattern'\t 'cardiomegaly'],66,M
4,sub-S03045_ses-E06910_acq-1_run-1_bp-chest_vp-...,sub-S03045,ses-E06910,['atypical pneumonia'\t ' consolidation'\t ' p...,66,M
5,sub-S03046_ses-E06217_acq-1_run-1_bp-chest_vp-...,sub-S03046,ses-E06217,['unchanged'],58,F


In [21]:
m2.reset_index(inplace=True,drop=True)

cn = ['filename','PatientID','ReportID','age','sex','Labels']
m2 = m2.reindex(columns=cn)

In [22]:
frames = [m2, pc2]
final_df = pd.concat(frames)

In [23]:
final_df['Labels2'] = final_df['Labels'].replace("'", '', regex=True)
final_df['Labels2'] = final_df['Labels'].replace('\t', ',', regex=True)
final_df['Labels2'] = final_df['Labels2'].replace("'", "", regex=True)
final_df['Labels2'] = final_df['Labels2'].replace('"', "", regex=True)
final_df['Labels2'] = final_df['Labels2'].str.replace(" ","")
final_df['Labels2'] = final_df['Labels2'].str.replace("[","")
final_df['Labels2'] = final_df['Labels2'].str.replace("]","")
final_df['Labels2'] = final_df['Labels2'].astype('category')

In [24]:
#final_df_dum= final_df.drop('Labels2', 1).join(final_df.Labels2.str.join('').str.get_dummies(','))
final_df_dum = final_df.Labels2.str.join('').str.get_dummies(',')

### Combine Datasets

In [25]:
df = pd.concat([final_df, final_df_dum], axis=1)

In [26]:
# add binary covid label
#covid_label = [1] * 2351 + [0]*1812
#df['covid_label'] = covid_label
m2.shape

(2351, 6)

In [27]:
df.age = df.age.astype('int')

In [28]:
#df.to_csv('E:/final_df.csv')

In [29]:
df.shape

(60941, 203)

In [30]:
df['filename']= df['filename'].str.replace(r'.png', '.jpg')


In [31]:
import copy
df2 = copy.copy(df)

#### Rename columns

In [32]:
df2.rename(columns={'radiologicalfinding':'radiological finding',
'differentialdiagnosis':'differential diagnosis',
'unchanged':'unchanged',
'obesity':'obesity',
'chronicchanges':'chronic changes',
'calcifieddensities':'calcified densities',
'granuloma':'granuloma',
'endonvessel':'end on vessel',
'adenopathy':'adenopathy',
'nodule':'nodule',
'pseudonodule':'pseudonodule',
'abscess':'abscess',
'cyst':'cyst',
'cavitation':'cavitation',
'fibroticband':'fibrotic band',
'volumeloss':'volume loss',
'hypoexpansion':'hypoexpansion',
'bullas':'bullas',
'pneumothorax':'pneumothorax',
'pneumoperitoneo':'pneumoperitoneo',
'pneumomediastinum':'pneumomediastinum',
'subcutaneousemphysema':'subcutaneous emphysema',
'hyperinflatedlung':'hyperinflated lung',
'flatteneddiaphragm':'flattened diaphragm',
'lungvascularpaucity':'lung vascular paucity',
'airtrapping':'air trapping',
'bronchiectasis':'bronchiectasis',
'infiltrates':'infiltrates',
'bronchovascularmarkings':'bronchovascular markings',
'airfluidlevel':'air fluid level',
'increaseddensity':'increased density',
'atelectasis':'atelectasis',
'mediastinalshift':'mediastinal shift',
'azygoslobe':'azygos lobe',
'fissurethickening':'fissure thickening',
'pleuralthickening':'pleural thickening',
'pleuralplaques':'pleural plaques',
'pleuraleffusion':'pleural effusion',
'pleuralmass':'pleural mass',
'costophrenicangleblunting':'costophrenic angle blunting',
'vascularredistribution':'vascular redistribution',
'hilarenlargement':'hilar enlargement',
'hilarcongestion':'hilar congestion',
'cardiomegaly':'cardiomegaly',
'pericardialeffusion':'pericardial effusion',
'kerleylines':'kerley lines',
'dextrocardia':'dextrocardia',
'rightsidedaorticarch':'right sided aortic arch',
'aorticatheromatosis':'aortic atheromatosis',
'aorticelongation':'aortic elongation',
'aorticaneurysm':'aortic aneurysm',
'mediastinalenlargement':'mediastinal enlargement',
'trachealshift':'tracheal shift',
'mass':'mass',
'esophagicdilatation':'esophagic dilatation',
'azygoesophagealrecessshift':'azygoesophageal recess shift',
'mediastiniclipomatosis':'mediastinic lipomatosis',
'thoraciccagedeformation':'thoracic cage deformation',
'vertebraldegenerativechanges':'vertebral degenerative changes',
'lyticbonelesion':'lytic bone lesion',
'scleroticbonelesion':'sclerotic bone lesion',
'costochondraljunctionhypertrophy':'costochondral junction hypertrophy',
'sternoclavicularjunctionhypertrophy':'sternoclavicular junction hypertrophy',
'axialhyperostosis':'axial hyperostosis',
'osteopenia':'osteopenia',
'osteoporosis':'osteoporosis',
'nonaxialarticulardegenerativechanges':'non axial articular degenerative changes',
'subacromialspacenarrowing':'subacromial space narrowing',
'fracture':'fracture',
'gynecomastia':'gynecomastia',
'hiatalhernia':'hiatal hernia',
'Chilaiditisign':'Chilaiditi sign',
'hemidiaphragmelevation':'hemidiaphragm elevation',
'diaphragmaticeventration':'diaphragmatic eventration',
'tracheostomytube':'tracheostomy tube',
'endotrachealtube':'endotracheal tube',
'NSGtube':'NSG tube',
'chestdraintube':'chest drain tube',
'ventriculoperitonealdraintube':'ventriculoperitoneal drain tube',
'gastrostomytube':'gastrostomy tube',
'nephrostomytube':'nephrostomy tube',
'doubleJstent':'double J stent',
'catheter':'catheter',
'electricaldevice':'electrical device',
'artificialheartvalve':'artificial heart valve',
'surgery':'surgery',
'abnormalforeignbody':'abnormal foreign body',
'externalforeignbody':'external foreign body',
'pneumonia':'pneumonia',
'tuberculosis':'tuberculosis',
'lungmetastasis':'lung metastasis',
'lymphangitiscarcinomatosa':'lymphangitis carcinomatosa',
'lepidicadenocarcinoma':'lepidic adenocarcinoma',
'pulmonaryfibrosis':'pulmonary fibrosis',
'emphysema':'emphysema',
'COPDsigns':'COPD signs',
'heartinsufficiency':'heart insufficiency',
'respiratorydistress':'respiratory distress',
'pulmonaryhypertension':'pulmonary hypertension',
'pulmonaryedema':'pulmonary edema',
'bonemetastasis':'bone metastasis',
'calcifiedgranuloma':'calcified granuloma',
'calcifiedadenopathy':'calcified adenopathy',
'calcifiedpleuralthickening':'calcified pleural thickening',
'calcifiedpleuralplaques':'calcified pleural plaques',
'heartvalvecalcified':'heart valve calcified',
'calcifiedfibroadenoma':'calcified fibroadenoma',
'multiplenodules':'multiple nodules',
'nippleshadow':'nipple shadow',
'hydropneumothorax':'hydropneumothorax',
'interstitialpattern':'interstitial pattern',
'alveolarpattern':'alveolar pattern',
'totalatelectasis':'total atelectasis',
'lobaratelectasis':'lobar atelectasis',
'segmentalatelectasis':'segmental atelectasis',
'laminaratelectasis':'laminar atelectasis',
'roundatelectasis':'round atelectasis',
'atelectasisbasal':'atelectasis basal',
'minorfissurethickening':'minor fissure thickening',
'majorfissurethickening':'major fissure thickening',
'loculatedfissuraleffusion':'loculated fissural effusion',
'apicalpleuralthickening':'apical pleural thickening',
'calcifiedpleuralthickening':'calcified pleural thickening',
'calcifiedpleuralplaques':'calcified pleural plaques',
'loculatedpleuraleffusion':'loculated pleural effusion',
'hydropneumothorax':'hydropneumothorax',
'empyema':'empyema',
'hemothorax':'hemothorax',
'centralvascularredistribution':'central vascular redistribution',
'adenopathy':'adenopathy',
'vascularhilarenlargement':'vascular hilar enlargement',
'descendentaorticelongation':'descendent aortic elongation',
'ascendentaorticelongation':'ascendent aortic elongation',
'aorticbuttonenlargement':'aortic button enlargement',
'supraaorticelongation':'supra aortic elongation',
'superiormediastinalenlargement':'superior mediastinal enlargement',
'mediastinalmass':'mediastinal mass',
'breastmass':'breast mass',
'pulmonarymass':'pulmonary mass',
'softtissuemass':'soft tissue mass',
'scoliosis':'scoliosis',
'kyphosis':'kyphosis',
'pectumexcavatum':'pectum excavatum',
'pectumcarinatum':'pectum carinatum',
'cervicalrib':'cervical rib',
'vertebralcompression':'vertebral compression',
'blasticbonelesion':'blastic bone lesion',
'claviclefracture':'clavicle fracture',
'humeralfracture':'humeral fracture',
'vertebralfracture':'vertebral fracture',
'ribfracture':'rib fracture',
'centralvenouscatheter':'central venous catheter',
'dualchamberdevice':'dual chamber device',
'singlechamberdevice':'single chamber device',
'pacemaker':'pacemaker',
'dai':'dai',
'artificialmitralheartvalve':'artificial mitral heart valve',
'artificialaorticheartvalve':'artificial aortic heart valve',
'metal':'metal',
'bonecement':'bone cement',
'prosthesis':'prosthesis',
'surgerybreast':'surgery breast',
'surgeryneck':'surgery neck',
'surgerylung':'surgery lung',
'surgeryheart':'surgery heart',
'surgeryhumeral':'surgery humeral',
'atypicalpneumonia':'atypical pneumonia ',
'viralpneumonia':'viral pneumonia',
'tuberculosissequelae':'tuberculosis sequelae',
'postradiotherapychanges':'post radiotherapy changes',
'asbestosissigns':'asbestosis signs',
'pulmonaryarteryhypertension':'pulmonary artery hypertension',
'pulmonaryvenoushypertension':'pulmonary venous hypertension',
'calcifiedmediastinaladenopathy':'calcified mediastinal adenopathy',
'groundglasspattern':'ground glass pattern',
'reticularinterstitialpattern':'reticular interstitial pattern',
'reticulonodularinterstitialpattern':'reticulonodular interstitial pattern',
'miliaryopacities':'miliary opacities',
'consolidation':'consolidation',
'airbronchogram':'air bronchogram',
'pulmonaryarteryenlargement':'pulmonary artery enlargement',
'goiter':'goiter',
'vertebralanteriorcompression':'vertebral anterior compression',
'callusribfracture':'callus rib fracture',
'centralvenouscatheterviasubclavianvein':'central venous catheter via subclavian vein',
'centralvenouscatheterviajugularvein':'central venous catheter via jugular vein',
'reservoircentralvenouscatheter':'reservoir central venous catheter',
'centralvenouscatheterviaumbilicalvein':'central venous catheter via umbilical vein',
'osteosynthesismaterial':'osteosynthesis material',
'sternotomy':'sternotomy',
'suturematerial':'suture material',
'humeralprosthesis':'humeral prosthesis',
'mammaryprosthesis':'mammary prosthesis',
'endoprosthesis':'endoprosthesis',
'mastectomy':'mastectomy',
'COVID19':'COVID19',
'COVID19uncertain':'COVID19 uncertain',
'aorticendoprosthesis':'aortic endoprosthesis'
}, inplace=True)

### Create hierarchical label structure

In [33]:
#level 5
df2.loc[df2['air bronchogram'] == 1, 'consolidation'] = 1
df2.loc[df2['aortic endoprosthesis'] == 1, 'endoprosthesis'] = 1

In [34]:
#level 4
df2.loc[df2['calcified mediastinal adenopathy'] == 1, 'calcified adenopathy'] = 1
df2.loc[df2['ground glass pattern'] == 1, 'interstitial pattern'] = 1
df2.loc[df2['reticular interstitial pattern'] == 1, 'interstitial pattern'] = 1
df2.loc[df2['reticulonodular interstitial pattern'] == 1, 'interstitial pattern'] = 1
df2.loc[df2['miliary opacities'] == 1, 'interstitial pattern'] = 1
df2.loc[df2['consolidation'] == 1, 'alveolar pattern'] = 1
df2.loc[df2['air bronchogram'] == 1, 'alveolar pattern'] = 1
df2.loc[df2['pulmonary artery enlargement'] == 1, 'vascular hilar enlargement'] = 1
df2.loc[df2['goiter'] == 1, 'superior mediastinal enlargement'] = 1
df2.loc[df2['supra aortic elongation'] == 1, 'superior mediastinal enlargement'] = 1
df2.loc[df2['vertebral anterior compression'] == 1, 'vertebral compression'] = 1
df2.loc[df2['callus rib fracture'] == 1, 'rib fracture'] = 1
df2.loc[df2['central venous catheter via subclavian vein'] == 1, 'central venous catheter'] = 1
df2.loc[df2['central venous catheter via jugular vein'] == 1, 'central venous catheter'] = 1
df2.loc[df2['reservoir central venous catheter'] == 1, 'central venous catheter'] = 1
#df2.loc[df2['central venous catheter via umbilical vein'] == 1, 'central venous catheter'] = 1
df2.loc[df2['osteosynthesis material'] == 1, 'metal'] = 1
df2.loc[df2['sternotomy'] == 1, 'metal'] = 1
df2.loc[df2['suture material'] == 1, 'metal'] = 1
df2.loc[df2['humeral prosthesis'] == 1, 'prosthesis'] = 1
df2.loc[df2['mammary prosthesis'] == 1, 'prosthesis'] = 1
df2.loc[df2['endoprosthesis'] == 1, 'prosthesis'] = 1
df2.loc[df2['mastectomy'] == 1, 'surgery breast'] = 1
df2.loc[df2['COVID19'] == 1, 'viral pneumonia'] = 1
df2.loc[df2['COVID19 uncertain'] == 1, 'viral pneumonia'] = 1


In [35]:
#level 3
df2.loc[df2['calcified granuloma'] == 1, 'calcified densities'] = 1
df2.loc[df2['calcified adenopathy'] == 1, 'calcified densities'] = 1
df2.loc[df2['calcified pleural thickening'] == 1, 'calcified densities'] = 1
df2.loc[df2['calcified pleural plaques'] == 1, 'calcified densities'] = 1
df2.loc[df2['heart valve calcified'] == 1, 'calcified densities'] = 1
df2.loc[df2['calcified fibroadenoma'] == 1, 'calcified densities'] = 1
df2.loc[df2['calcified fibroadenoma'] == 1, 'granuloma'] = 1
df2.loc[df2['calcified granuloma'] == 1, 'granuloma'] = 1
df2.loc[df2['calcified adenopathy'] == 1, 'adenopathy'] = 1
df2.loc[df2['multiple nodules'] == 1, 'nodule'] = 1
df2.loc[df2['nipple shadow'] == 1, 'pseudonodule'] = 1
df2.loc[df2['end on vessel'] == 1, 'pseudonodule'] = 1
df2.loc[df2['hydropneumothorax'] == 1, 'pneumothorax'] = 1
df2.loc[df2['interstitial pattern'] == 1, 'infiltrates'] = 1
df2.loc[df2['alveolar pattern'] == 1, 'infiltrates'] = 1
df2.loc[df2['total atelectasis'] == 1, 'atelectasis'] = 1
df2.loc[df2['lobar atelectasis'] == 1, 'atelectasis'] = 1
df2.loc[df2['segmental atelectasis'] == 1, 'atelectasis'] = 1
df2.loc[df2['laminar atelectasis'] == 1, 'atelectasis'] = 1
df2.loc[df2['round atelectasis'] == 1, 'atelectasis'] = 1
df2.loc[df2['atelectasis basal'] == 1, 'atelectasis'] = 1
df2.loc[df2['minor fissure thickening'] == 1, 'fissure thickening'] = 1
df2.loc[df2['major fissure thickening'] == 1, 'fissure thickening'] = 1
df2.loc[df2['loculated fissural effusion'] == 1, 'fissure thickening'] = 1
df2.loc[df2['apical pleural thickening'] == 1, 'pleural thickening'] = 1
df2.loc[df2['calcified pleural thickening'] == 1, 'pleural thickening'] = 1
df2.loc[df2['calcified pleural plaques'] == 1, 'pleural plaques'] = 1
df2.loc[df2['loculated pleural effusion'] == 1, 'pleural effusion'] = 1
df2.loc[df2['loculated fissural effusion'] == 1, 'pleural effusion'] = 1
df2.loc[df2['hydropneumothorax'] == 1, 'pleural effusion'] = 1
df2.loc[df2['empyema'] == 1, 'pleural effusion'] = 1
#df2.loc[df2['hemothorax'] == 1, 'pleural effusion'] = 1
df2.loc[df2['central vascular redistribution'] == 1, 'vascular redistribution'] = 1
df2.loc[df2['adenopathy'] == 1, 'hilar enlargement'] = 1
df2.loc[df2['vascular hilar enlargement'] == 1, 'hilar enlargement'] = 1
df2.loc[df2['descendent aortic elongation'] == 1, 'aortic elongation'] = 1
df2.loc[df2['ascendent aortic elongation'] == 1, 'aortic elongation'] = 1
df2.loc[df2['aortic button enlargement'] == 1, 'aortic elongation'] = 1
df2.loc[df2['supra aortic elongation'] == 1, 'aortic elongation'] = 1
df2.loc[df2['superior mediastinal enlargement'] == 1, 'mediastinal enlargement'] = 1
df2.loc[df2['descendent aortic elongation'] == 1, 'mediastinal enlargement'] = 1
df2.loc[df2['ascendent aortic elongation'] == 1, 'mediastinal enlargement'] = 1
df2.loc[df2['aortic aneurysm'] == 1, 'mediastinal enlargement'] = 1
df2.loc[df2['mediastinal mass'] == 1, 'mediastinal enlargement'] = 1
df2.loc[df2['hiatal hernia'] == 1, 'mediastinal enlargement'] = 1
df2.loc[df2['mediastinal mass'] == 1, 'mass'] = 1
df2.loc[df2['breast mass'] == 1, 'mass'] = 1
df2.loc[df2['pleural mass'] == 1, 'mass'] = 1
df2.loc[df2['pulmonary mass'] == 1, 'mass'] = 1
df2.loc[df2['soft tissue mass'] == 1, 'mass'] = 1
df2.loc[df2['scoliosis'] == 1, 'thoracic cage deformation'] = 1
df2.loc[df2['kyphosis'] == 1, 'thoracic cage deformation'] = 1
df2.loc[df2['pectum excavatum'] == 1, 'thoracic cage deformation'] = 1
df2.loc[df2['pectum carinatum'] == 1, 'thoracic cage deformation'] = 1
df2.loc[df2['cervical rib'] == 1, 'thoracic cage deformation'] = 1
df2.loc[df2['vertebral compression'] == 1, 'vertebral degenerative changes'] = 1
df2.loc[df2['blastic bone lesion'] == 1, 'sclerotic bone lesion'] = 1
df2.loc[df2['clavicle fracture'] == 1, 'fracture'] = 1
df2.loc[df2['humeral fracture'] == 1, 'fracture'] = 1
df2.loc[df2['vertebral fracture'] == 1, 'fracture'] = 1
df2.loc[df2['rib fracture'] == 1, 'fracture'] = 1
df2.loc[df2['central venous catheter'] == 1, 'catheter'] = 1
df2.loc[df2['dual chamber device'] == 1, 'electrical device'] = 1
df2.loc[df2['single chamber device'] == 1, 'electrical device'] = 1
df2.loc[df2['pacemaker'] == 1, 'electrical device'] = 1
df2.loc[df2['dai'] == 1, 'electrical device'] = 1
df2.loc[df2['artificial mitral heart valve'] == 1, 'artificial heart valve'] = 1
df2.loc[df2['artificial aortic heart valve'] == 1, 'artificial heart valve'] = 1
df2.loc[df2['metal'] == 1, 'surgery'] = 1
df2.loc[df2['bone cement'] == 1, 'surgery'] = 1
df2.loc[df2['prosthesis'] == 1, 'surgery'] = 1
df2.loc[df2['surgery breast'] == 1, 'surgery'] = 1
df2.loc[df2['surgery neck'] == 1, 'surgery'] = 1
df2.loc[df2['surgery lung'] == 1, 'surgery'] = 1
df2.loc[df2['surgery heart'] == 1, 'surgery'] = 1
df2.loc[df2['surgery humeral'] == 1, 'surgery'] = 1
#df2.loc[df2['atypical pneumonia'] == 1, 'pneumonia'] = 1
df2.loc[df2['viral pneumonia'] == 1, 'pneumonia'] = 1
df2.loc[df2['tuberculosis sequelae'] == 1, 'tuberculosis'] = 1
df2.loc[df2['post radiotherapy changes'] == 1, 'pulmonary fibrosis'] = 1
df2.loc[df2['asbestosis signs'] == 1, 'pulmonary fibrosis'] = 1
df2.loc[df2['pulmonary artery hypertension'] == 1, 'pulmonary hypertension'] = 1
df2.loc[df2['pulmonary venous hypertension'] == 1, 'pulmonary hypertension'] = 1


In [36]:
#level 2
df2.loc[df2['unchanged'] == 1, 'radiological finding'] = 1
df2.loc[df2['obesity'] == 1, 'radiological finding'] = 1
df2.loc[df2['chronic changes'] == 1, 'radiological finding'] = 1
df2.loc[df2['calcified densities'] == 1, 'radiological finding'] = 1
df2.loc[df2['granuloma'] == 1, 'radiological finding'] = 1
df2.loc[df2['end on vessel'] == 1, 'radiological finding'] = 1
df2.loc[df2['adenopathy'] == 1, 'radiological finding'] = 1
df2.loc[df2['nodule'] == 1, 'radiological finding'] = 1
df2.loc[df2['pseudonodule'] == 1, 'radiological finding'] = 1
df2.loc[df2['abscess'] == 1, 'radiological finding'] = 1
df2.loc[df2['cyst'] == 1, 'radiological finding'] = 1
df2.loc[df2['cavitation'] == 1, 'radiological finding'] = 1
df2.loc[df2['fibrotic band'] == 1, 'radiological finding'] = 1
df2.loc[df2['volume loss'] == 1, 'radiological finding'] = 1
df2.loc[df2['hypoexpansion'] == 1, 'radiological finding'] = 1
df2.loc[df2['bullas'] == 1, 'radiological finding'] = 1
df2.loc[df2['pneumothorax'] == 1, 'radiological finding'] = 1
df2.loc[df2['pneumoperitoneo'] == 1, 'radiological finding'] = 1
df2.loc[df2['pneumomediastinum'] == 1, 'radiological finding'] = 1
df2.loc[df2['subcutaneous emphysema'] == 1, 'radiological finding'] = 1
df2.loc[df2['hyperinflated lung'] == 1, 'radiological finding'] = 1
df2.loc[df2['flattened diaphragm'] == 1, 'radiological finding'] = 1
df2.loc[df2['lung vascular paucity'] == 1, 'radiological finding'] = 1
df2.loc[df2['air trapping'] == 1, 'radiological finding'] = 1
df2.loc[df2['bronchiectasis'] == 1, 'radiological finding'] = 1
df2.loc[df2['infiltrates'] == 1, 'radiological finding'] = 1
df2.loc[df2['bronchovascular markings'] == 1, 'radiological finding'] = 1
df2.loc[df2['air fluid level'] == 1, 'radiological finding'] = 1
df2.loc[df2['increased density'] == 1, 'radiological finding'] = 1
df2.loc[df2['atelectasis'] == 1, 'radiological finding'] = 1
df2.loc[df2['mediastinal shift'] == 1, 'radiological finding'] = 1
df2.loc[df2['azygos lobe'] == 1, 'radiological finding'] = 1
df2.loc[df2['fissure thickening'] == 1, 'radiological finding'] = 1
df2.loc[df2['pleural thickening'] == 1, 'radiological finding'] = 1
df2.loc[df2['pleural plaques'] == 1, 'radiological finding'] = 1
df2.loc[df2['pleural effusion'] == 1, 'radiological finding'] = 1
df2.loc[df2['pleural mass'] == 1, 'radiological finding'] = 1
df2.loc[df2['costophrenic angle blunting'] == 1, 'radiological finding'] = 1
df2.loc[df2['vascular redistribution'] == 1, 'radiological finding'] = 1
df2.loc[df2['hilar enlargement'] == 1, 'radiological finding'] = 1
df2.loc[df2['hilar congestion'] == 1, 'radiological finding'] = 1
df2.loc[df2['cardiomegaly'] == 1, 'radiological finding'] = 1
df2.loc[df2['pericardial effusion'] == 1, 'radiological finding'] = 1
df2.loc[df2['kerley lines'] == 1, 'radiological finding'] = 1
df2.loc[df2['dextrocardia'] == 1, 'radiological finding'] = 1
df2.loc[df2['right sided aortic arch'] == 1, 'radiological finding'] = 1
df2.loc[df2['aortic atheromatosis'] == 1, 'radiological finding'] = 1
df2.loc[df2['aortic elongation'] == 1, 'radiological finding'] = 1
df2.loc[df2['aortic aneurysm'] == 1, 'radiological finding'] = 1
df2.loc[df2['mediastinal enlargement'] == 1, 'radiological finding'] = 1
df2.loc[df2['tracheal shift'] == 1, 'radiological finding'] = 1
df2.loc[df2['mass'] == 1, 'radiological finding'] = 1
df2.loc[df2['esophagic dilatation'] == 1, 'radiological finding'] = 1
df2.loc[df2['azygoesophageal recess shift'] == 1, 'radiological finding'] = 1
df2.loc[df2['mediastinic lipomatosis'] == 1, 'radiological finding'] = 1
df2.loc[df2['thoracic cage deformation'] == 1, 'radiological finding'] = 1
df2.loc[df2['vertebral degenerative changes'] == 1, 'radiological finding'] = 1
df2.loc[df2['lytic bone lesion'] == 1, 'radiological finding'] = 1
df2.loc[df2['sclerotic bone lesion'] == 1, 'radiological finding'] = 1
df2.loc[df2['costochondral junction hypertrophy'] == 1, 'radiological finding'] = 1
df2.loc[df2['sternoclavicular junction hypertrophy'] == 1, 'radiological finding'] = 1
df2.loc[df2['axial hyperostosis'] == 1, 'radiological finding'] = 1
df2.loc[df2['osteopenia'] == 1, 'radiological finding'] = 1
df2.loc[df2['osteoporosis'] == 1, 'radiological finding'] = 1
df2.loc[df2['non axial articular degenerative changes'] == 1, 'radiological finding'] = 1
df2.loc[df2['subacromial space narrowing'] == 1, 'radiological finding'] = 1
df2.loc[df2['fracture'] == 1, 'radiological finding'] = 1
df2.loc[df2['gynecomastia'] == 1, 'radiological finding'] = 1
df2.loc[df2['hiatal hernia'] == 1, 'radiological finding'] = 1
df2.loc[df2['Chilaiditi sign'] == 1, 'radiological finding'] = 1
df2.loc[df2['hemidiaphragm elevation'] == 1, 'radiological finding'] = 1
df2.loc[df2['diaphragmatic eventration'] == 1, 'radiological finding'] = 1
df2.loc[df2['tracheostomy tube'] == 1, 'radiological finding'] = 1
df2.loc[df2['endotracheal tube'] == 1, 'radiological finding'] = 1
df2.loc[df2['NSG tube'] == 1, 'radiological finding'] = 1
df2.loc[df2['chest drain tube'] == 1, 'radiological finding'] = 1
df2.loc[df2['ventriculoperitoneal drain tube'] == 1, 'radiological finding'] = 1
df2.loc[df2['gastrostomy tube'] == 1, 'radiological finding'] = 1
df2.loc[df2['nephrostomy tube'] == 1, 'radiological finding'] = 1
df2.loc[df2['double J stent'] == 1, 'radiological finding'] = 1
df2.loc[df2['catheter'] == 1, 'radiological finding'] = 1
df2.loc[df2['electrical device'] == 1, 'radiological finding'] = 1
df2.loc[df2['artificial heart valve'] == 1, 'radiological finding'] = 1
df2.loc[df2['surgery'] == 1, 'radiological finding'] = 1
df2.loc[df2['abnormal foreign body'] == 1, 'radiological finding'] = 1
df2.loc[df2['external foreign body'] == 1, 'radiological finding'] = 1
df2.loc[df2['pneumonia'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['tuberculosis'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['lung metastasis'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['lymphangitis carcinomatosa'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['lepidic adenocarcinoma'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['pulmonary fibrosis'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['emphysema'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['COPD signs'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['heart insufficiency'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['respiratory distress'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['pulmonary hypertension'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['pulmonary edema'] == 1, 'differential diagnosis'] = 1
df2.loc[df2['bone metastasis'] == 1, 'differential diagnosis'] = 1


### Create train/testvalidation datasets

In [37]:
#load train_val file list

train_val = 'E:\\osfstorage-archive\\C19_images\\final_jpg\\'
tv_list = list()
import os
for root, dirs, files in os.walk(train_val):
    for name in files:
        if name.endswith((".jpg")):
            tv_list.append(name)
            
len(tv_list)

40845

In [38]:
test = 'E:\\osfstorage-archive\\C19_images\\test\\'
test_list = list()
import os
for root, dirs, files in os.walk(test):
    for name in files:
        if name.endswith((".jpg")):
            test_list.append(name)
len(test_list)  

11493

In [52]:
test_list_df = pd.DataFrame(test_list)
test_list_df.to_csv('E:/test_list_toomany.csv',index=False)

In [39]:
df_trainval = df2[df2['filename'].isin(tv_list)]

In [40]:
df_trainval['radiological finding'].fillna(0, inplace=True)
df_trainval['differential diagnosis'].fillna(0, inplace=True)

C:\Users\colmc\Anaconda3\envs\new_env\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [41]:
df_trainval.to_csv('E:/df_train_val.csv')

In [42]:
df_trainval_2 = df_trainval[df_trainval['unchanged'] == 0]
df_trainval_2.drop(['unchanged'], axis=1, inplace=True)

In [43]:
#drop images with no labels
filter2 = df_trainval_2["Labels2"] != ""
df_trainval_2 = df_trainval_2[filter2]


In [44]:
df_trainval_2.to_csv('E:/df_train_val_unchanged.csv')

In [45]:
df_trainval_sort = df_trainval_2.sort_values('PatientID').reset_index(drop=True)

df_train = pd.concat([df_trainval_sort.head(29500), df_trainval_sort.tail(1090)])
df_val = df_trainval_sort[~df_trainval_sort.isin(df_train)].dropna()
df_val = df_val.reset_index(drop=True) 
df_train = df_train.reset_index(drop=True)

In [46]:
df_train.to_csv('E:/train.csv')
df_val.to_csv('E:/validate.csv')

In [47]:
#remove suboptimal,exclude?

In [49]:
df_test = df2[df2['filename'].isin(test_list)]
df_test['radiological finding'].fillna(0, inplace=True)
df_test['differential diagnosis'].fillna(0, inplace=True)
df_test_2 = df_test[df_test['unchanged'] == 0]
df_test_2.drop(['unchanged'], axis=1, inplace=True)
#drop images with no labels
filter2 = df_test_2["Labels2"] != ""
df_test_2 = df_test_2[filter2]
df_test_2.to_csv('E:/test_2.csv')

In [50]:
df_test_2.shape

(10158, 204)